In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.6 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.6 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv("/content/drive/MyDrive/Datasets/BanglaSongLyrics.csv")
train.head()

,title,category,lyrics
0,অপরাজিত,ব্যান্ড,এবার তোমার কথাগুলোর\nকাছে আমি অপরাজিত\nজমে থাক...
1,আমার ভায়ের রক্তে রাঙানো,দেশাত্মবোধক গান,আমার ভায়ের রক্তে রাঙানো একুশে ফেব্রুয়ারি\nআমি ...
2,দয়াল চান্দের পায়ের দুলা আমি যেন হইতে পারি,বাউল,আমি কুকুর হইয়া দেই পাহাড়া\nদয়াল চান্দের ঘড় বাড়...
3,মাওলানা,বাউল,তুমি এসমে আজ যত তরানে ওলা\nগাউছেলাম বাবা নুরে ...
4,আমার আপনার চেয়ে আপন যে জন,নজরুল গীতি,আমার আপনার চেয়ে আপন যে জন\nখুঁজি তারে আমি আপন...


In [5]:
del train['title']

In [6]:
train.drop(0, inplace=True, axis=0)
# test.drop(0, inplace=True, axis=0)
labels = list(set(train['category']))
print(labels)

['আধুনিক', 'নজরুল গীতি', 'রবীন্দ্র সংগীত', 'নাটক', 'Uncategorized', 'পপ সঙ্গীত', 'জীবনমুখী গান', 'ভক্তিমূলক গান', 'লালন', 'র\u200d্যাপ', 'ইসলামী গান', 'দেশাত্মবোধক গান', 'বিবিধ', 'পল্লীগীতি', 'ভাটিয়ালি', 'হাসন রাজার গান', 'বাউল', 'ছড়াগান', 'ছায়াছবি', 'ভাওয়াইয়া গান', 'ব্যান্ড']


In [7]:
len(labels)

21

In [8]:
train['category'] = train['category'].replace({'আধুনিক':0, 'নাটক':1, 'বাউল':2, 'ইসলামী গান':3, 'পপ সঙ্গীত':4, 'ভাওয়াইয়া গান':5, 'ছড়াগান':6, 'দেশাত্মবোধক গান':7, 'Uncategorized':8, 'পল্লীগীতি':9, 'হাসন রাজার গান':10, 'ছায়াছবি':11, 'ভাটিয়ালি':12, 'জীবনমুখী গান':13, 'নজরুল গীতি':14, 'র\u200d্যাপ':15, 'রবীন্দ্র সংগীত':16, 'ব্যান্ড':17, 'ভক্তিমূলক গান':18, 'লালন':19, 'বিবিধ':20})
# test['class_label'] = test['class_label'].replace({'negative':0, 'positive':1, 'neutral':2})

In [9]:
labels = list(set(train['category']))
print(labels)
# del train['id']
train.head()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


,category,lyrics
1,7,আমার ভায়ের রক্তে রাঙানো একুশে ফেব্রুয়ারি\nআমি ...
2,2,আমি কুকুর হইয়া দেই পাহাড়া\nদয়াল চান্দের ঘড় বাড়...
3,2,তুমি এসমে আজ যত তরানে ওলা\nগাউছেলাম বাবা নুরে ...
4,14,আমার আপনার চেয়ে আপন যে জন\nখুঁজি তারে আমি আপন...
5,14,বোলো না বোলো না ওলো সই\nআর সে কথা।।\nতরু কি লত...


In [10]:
# del test['id']
# test.head()

In [11]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train, test_size=.2)

In [12]:
train_texts = train_data['lyrics'].values.tolist()
train_labels = train_data['category'].values.tolist()
test_texts = test_data['lyrics'].values.tolist()
test_labels = test_data['category'].values.tolist()

In [13]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import Trainer,TrainingArguments

In [14]:
tokenizer = AutoTokenizer.from_pretrained('csebuetnlp/banglabert',num_labels=21)

Downloading:   0%|          | 0.00/119 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/586 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [15]:
train_encodings = tokenizer(train_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')
test_encodings = tokenizer(test_texts, truncation=True, max_length=512,padding=True,return_tensors = 'pt')

In [16]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
## Test Dataset
class SentimentTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

In [17]:
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [18]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    y_score = pred
    y_true = labels

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true, y_score, average='weighted')
    f1 = f1_score(labels, pred, average='weighted')

    return {"accuracy": accuracy,"f1_score":f1, "recall": recall, "precision": precision}

In [19]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Datasets/CSE495/result',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True,
)

In [20]:
model = AutoModelForSequenceClassification.from_pretrained('csebuetnlp/banglabert',num_labels=21)

# for name, param in model.named_parameters():
#      if name.startswith("bert.encoder.layer.1"): # choose whatever you like here
#         param.requires_grad = False

trainer = Trainer(
    model=model,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=test_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)


Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.weight', 'classifie

In [21]:
trainer.train()

***** Running training *****
  Num examples = 3283
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8220


Step,Training Loss,Validation Loss,Accuracy,F1 Score,Recall,Precision
500,2.004000,1.524567,0.514007,0.454604,0.514007,0.424723


***** Running Evaluation *****
  Num examples = 821
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Datasets/CSE495/result/checkpoint-500
Configuration saved in /content/drive/MyDrive/Datasets/CSE495/result/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Datasets/CSE495/result/checkpoint-500/pytorch_model.bin


KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()